In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tvDatafeed import TvDatafeed, Interval
from config.account_info import *

tv = TvDatafeed(USERNAME, PASSWARD)

error while signin
you are using nologin method, data you access may be limited


In [12]:
MONTH_CODE = {
    1:"F", 2:"G", 3:"H", 4:"J", 5:"K", 6:"M",
    7:"N", 8:"Q", 9:"U", 10:"V", 11:"X", 12:"Z"
}

def get_futures_symbol_for_fomc(meeting_date : pd.Timestamp) -> str:
    """
    FOMC 회의 날짜로부터 해당 월의 ZQ 선물 심볼을 생성합니다.
    예: 2025-03-18 → ZQH2025
    """
    if not isinstance(meeting_date, pd.Timestamp):
        meeting_date = pd.to_datetime(meeting_date)

    year = meeting_date.year
    month = meeting_date.month
    mcode = MONTH_CODE[month]
    return f"ZQ{mcode}{year}"

In [54]:
def get_interest_probs(
        futures_prices: pd.Series,
        current_rate: float,
        bp_moves: list[float] | np.ndarray,
        meeting_date: pd.Timestamp | str,
        convexity_adj : float = 0.0
    ) -> pd.DataFrame:
    """
    ZQ 월물 선물 종가 시계열로부터,
    임의의 bp 변동 시나리오들에 대한 확률 시계열을 계산합니다.
    (CME FedWatch 스타일: '월평균금리 = 시나리오별 월평균의 확률가중평균' 가정)

    Parameters
    ----------
    futures_prices : pd.Series
        index = DatetimeIndex, values = 선물 종가 (price)
    current_rate : float
        회의 직전 기준금리 (예: 4.875, target range 중앙값 기준)
    bp_moves : list[float] or np.ndarray
        회의에서 변동될 bp 단위 리스트
        예: [-50, -25, 0, 25, 50]
    meeting_date : pd.Timestamp
        해당 FOMC 회의 날짜

    Returns
    -------
    pd.DataFrame
        columns:
            - 'price'      : 선물 가격
            - 'impl_rate'  : 선물에서 역산한 월평균 EFFR
            - 'prob_[bp]'  : 각 bp 시나리오 확률 (예: prob_-25, prob_0, prob_25 ...)
    """
    if not isinstance(meeting_date, pd.Timestamp):
        meeting_date = pd.to_datetime(meeting_date)

    # 월 시작과 끝
    month_start = meeting_date.replace(day=1)
    month_end = (month_start + pd.offsets.MonthEnd(0))

    # 월 전체 일수
    N = (month_end - month_start).days + 1

    # 회의 다음 영업일 계산
    def next_bday(d):
        d = pd.to_datetime(d)
        d += pd.Timedelta(days=1)
        while d.weekday() >= 5:  # 토/일 제외
            d += pd.Timedelta(days=1)
        return d

    first_after = next_bday(meeting_date)

    # 회의 다음 영업일이 이번달 밖이면 → 이번달 전체 r0 유지
    if first_after > month_end:
        N_before = N
        N_after = 0
    else:
        # month_start ~ (first_after-1)까지 r0
        N_before = (first_after - month_start).days
        N_after = N - N_before                     # 회의 다음 날부터 월말까지

    r0 = current_rate

    # bp_moves → post-meeting rate → 해당 월 평균 금리
    bp_moves = np.asarray(bp_moves, dtype=float)

    # bp를 %로 변환: 25bp -> 0.25, 50bp -> 0.50
    delta_rates = bp_moves / 100.0

    post_rates = r0 + delta_rates  # 회의 후 기준금리
    avg_rates = (N_before * r0 + N_after * post_rates) / N  # 시나리오별 월평균 금리

    # avg_rates 기준으로 정렬 (아래에서 이웃한 두 시나리오 보간에 사용)
    order = np.argsort(avg_rates)
    avg_sorted = avg_rates[order]

    prices = futures_prices.sort_index()
    impl_rates = (100.0 - prices.values) + convexity_adj # 선물에서 역산한 월평균 EFFR

    probs_matrix = []

    for r_impl in impl_rates:
        # 각 시나리오 확률 (정렬된 순서)
        probs = np.zeros_like(avg_sorted)

        # 1) r_impl이 최솟값보다 작거나 같으면 → 최솟값 시나리오 확률 1
        if r_impl <= avg_sorted[0]:
            probs[0] = 1.0

        # 2) r_impl이 최댓값보다 크거나 같으면 → 최댓값 시나리오 확률 1
        elif r_impl >= avg_sorted[-1]:
            probs[-1] = 1.0

        else:
            # 3) 그 사이면, 인접 구간 찾아서 두 시나리오만 선형보간
            #    avg_sorted[i] <= r_impl <= avg_sorted[i+1] 인 i 찾기
            idx = np.searchsorted(avg_sorted, r_impl) - 1
            idx = max(0, min(idx, len(avg_sorted) - 2))

            r_low = avg_sorted[idx]
            r_high = avg_sorted[idx + 1]

            denom = (r_high - r_low)
            if abs(denom) < 1e-10:
                # 두 시나리오 월평균이 거의 같으면 아래쪽에 몰아주기
                probs[idx] = 1.0
            else:
                # r_impl이 r_low에 가까우면 low 확률↑, r_high에 가까우면 high 확률↑
                w_high = (r_impl - r_low) / denom
                w_low = 1.0 - w_high
                probs[idx] = w_low
                probs[idx + 1] = w_high

        # 혹시 numerical noise 있으면 정규화
        s = probs.sum()
        if s > 0:
            probs /= s

        probs_matrix.append(probs)

    probs_matrix = np.array(probs_matrix)

    # 정렬 전 원래 bp 순서로 되돌리기
    inv_order = np.argsort(order)
    probs_original_order = probs_matrix[:, inv_order]

    # 컬럼 이름 만들기: prob_-25, prob_0, prob_25 ...
    prob_cols = {
        f"prob_{int(bp)}": probs_original_order[:, i]
        for i, bp in enumerate(bp_moves)
    }

    out = pd.DataFrame(
        {
            "price": prices.values,
            "impl_rate": impl_rates,
            **prob_cols,
        },
        index=prices.index,
    )

    return out

In [55]:
zq_series = tv.get_hist(
    symbol='ZQZ2025',
    exchange='CBOT', # Index Exchange
    interval=Interval.in_daily,
    n_bars=2000
)
zq_series.index = pd.to_datetime(zq_series.index.strftime('%Y-%m-%d'))

In [78]:
effr_series = tv.get_hist(
    symbol='EFFR',
    exchange='FRED', # Index Exchange
    interval=Interval.in_daily,
    n_bars=10000
)
effr_series.index = pd.to_datetime(effr_series.index.strftime('%Y-%m-%d'))

In [80]:
effr_series['close']

datetime
2000-07-03    7.03
2000-07-05    6.52
2000-07-06    6.51
2000-07-07    6.42
2000-07-10    6.51
              ... 
2025-11-18    3.88
2025-11-19    3.88
2025-11-20    3.88
2025-11-21    3.88
2025-11-24    3.88
Name: close, Length: 6376, dtype: float64

In [56]:
zq_series['close']

datetime
2020-12-31    99.860
2021-01-04    99.870
2021-01-05    99.870
2021-01-06    99.855
2021-01-07    99.845
               ...  
2025-11-20    96.175
2025-11-21    96.220
2025-11-24    96.245
2025-11-25    96.255
2025-11-26    96.255
Name: close, Length: 1235, dtype: float64

In [59]:
def calibrate_convexity_adj(
    futures_prices: pd.Series,
    current_rate: float,
    bp_moves: list[float] | np.ndarray,
    meeting_date: pd.Timestamp | str,
    target_prob: float,   # FedWatch에서 본 컷 확률 (예: 84.3)
    target_bp: float,     # 그 컷이 몇 bp인지 (예: -25)
    effr: float | None = None,
    c_min: float = -0.05,
    c_max: float = 0.05,
    n_grid: int = 201,
) -> tuple[float, float]:
    """
    FedWatch 확률(target_prob)에 가장 근접하도록
    convexity_adj 상수 c 를 calibration 합니다.

    반환:
        best_c   : 최적 convexity_adj (%)
        best_err : FedWatch 대비 절대 오차 (%p)
    """
    bp_moves = np.asarray(bp_moves, dtype=float)
    if effr is not None:
        r0 = effr
    else:
        r0 = current_rate

    grid = np.linspace(c_min, c_max, n_grid)

    best_c = 0.0
    best_err = np.inf

    target_col = f"prob_{int(target_bp)}"

    for c in grid:
        df = get_interest_probs(
            futures_prices=futures_prices,
            current_rate=r0,
            bp_moves=bp_moves,
            meeting_date=meeting_date,
            convexity_adj=c,
        )
        p = df[target_col].iloc[-1] * 100.0  # 모델 확률 (%로)
        err = abs(p - target_prob)

        if err < best_err:
            best_err = err
            best_c = c

    return best_c, best_err

In [60]:
best_c, best_err = calibrate_convexity_adj(
    futures_prices = zq_series['close'],
    current_rate = 3.875,
    bp_moves = [-75, -50, -25, 0, 25, 50, 75],
    meeting_date = "2025-12-10",
    target_prob = 84.3,    # FedWatch 컷 확률 (%)
    target_bp = -25,     # -25bp 컷
    effr = 3.88,    # 실제 EFFR (있으면 넣고, 없으면 None)
)

In [61]:
best_c

np.float64(-0.008)

In [62]:
best_err

np.float64(0.13809523809246116)

In [74]:
df_probs = get_interest_probs(
    futures_prices = zq_series['close'],        # ZQH2025 같은 월물 종가 Series
    current_rate = 3.89,          # 회의 직전 기준금리
    bp_moves = [-75, -50, -25, 0, 25, 50, 75],
    meeting_date = "2025-12-10",
    convexity_adj = 0
)

In [75]:
df_probs.tail(10)

,price,impl_rate,prob_-75,prob_-50,prob_-25,prob_0,prob_25,prob_50,prob_75
datetime,,,,,,,,,
2025-11-13,96.210,3.790,0.0,0.0,0.590476,0.409524,0.0,0.0,0.0
2025-11-14,96.195,3.805,0.0,0.0,0.501905,0.498095,0.0,0.0,0.0
2025-11-17,96.185,3.815,0.0,0.0,0.442857,0.557143,0.0,0.0,0.0
2025-11-18,96.195,3.805,0.0,0.0,0.501905,0.498095,0.0,0.0,0.0
2025-11-19,96.165,3.835,0.0,0.0,0.324762,0.675238,0.0,0.0,0.0
2025-11-20,96.175,3.825,0.0,0.0,0.383810,0.616190,0.0,0.0,0.0
2025-11-21,96.220,3.780,0.0,0.0,0.649524,0.350476,0.0,0.0,0.0
2025-11-24,96.245,3.755,0.0,0.0,0.797143,0.202857,0.0,0.0,0.0
2025-11-25,96.255,3.745,0.0,0.0,0.856190,0.143810,0.0,0.0,0.0
